# Introduction to pandas

[pandas](https://pandas.pydata.org/docs/) is a Python package for working with structured datasets, e.g. it is perfectly suited for observational/statistical datasets, having many similarities with Excel spreadsheets.

## Key features
- easy handling of missing data
- columns can be inserted and deleted from loaded data sets (size mutability)
- data can be automatically or explicitly aligned to a set of labels (data alignment)
- group by functionality to perform split-apply-combine operations on data sets
- easy to convert ragged, differently-indexed data in other Python and NumPy data structures into pandas objects
- intelligent label-based slicing, fancy indexing, and subsetting of large data sets
- intuitive merging and joining data sets
- flexible reshaping and pivoting of data sets
- hierarchical labeling of axes (possible to have multiple labels per tick)
- robust I/O (Input/Output) tools for loading data from flat files (CSV and delimited), Excel files, databases, and saving/loading data from the ultrafast HDF5 format
- time series - specific functionalities

pandas is built on top of [`numpy`](https://numpy.org/doc/stable/index.html) and is intended to integrate well within a scientific computing environment with many other 3rd party libraries.

The community standard to import pandas is to use `pd` alias:

In [ ]:
import pandas as pd

# Import other useful modules
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Primary data structures of pandas

### Series

Series is a 1-dimensional numpy array with axis labels. Can be created with a dictionary:

In [ ]:
d = {'a': 1, 'b': 2, 'c': 3}
ser = pd.Series(data=d, index=['a', 'b', 'c'])
print(ser)

### DataFrame 

DataFrame is a 2-dimensional tabular data. Similarly, can be created with a dictionary:

In [ ]:
d = {'temperature': [31.5, 32.5], 'pressure': [100.0, 200.0]}
df = pd.DataFrame(data=d)
print(df)

## Loading data

The following cell finds the path to the data on your system so we can read it in:

In [ ]:
from pathlib import Path # A module for dealing with paths that is operating system agnostic
notebook_dir = Path.cwd() # Get the current working directory
base_dir = notebook_dir.parent.absolute() # Get the parent directory (folder that contains our working directory)
data_dir = base_dir / 'data'# Get diretory where the data is stored
fname = data_dir / 'ship_ctd_short.csv'

Data from a `.csv` file can be loaded using [`pandas.read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function. For other data formats, check how to load them [here](https://pandas.pydata.org/pandas-docs/stable/reference/io.html).

In [ ]:
ctd_data = pd.read_csv(fname)

Let's interrogate `ctd_data` variable:

In [ ]:
type(ctd_data)

In [ ]:
# View just the top of data
ctd_data.head(6)

In [ ]:
# Find shape of the data
print(ctd_data.shape)

In [ ]:
# Find data type in each column
print(ctd_data.dtypes)

In [ ]:
# View the last rows of data
# Note the optional argument for number of rows (available for head() too)
ctd_data.tail(n=2) 

Get descriptors for the **vertical** axis (rows):

In [ ]:
print(ctd_data.index)

Get descriptors for the **horizontal** axis (columns):

In [ ]:
print(ctd_data.columns)

Get general information at once including memory usage:

In [ ]:
ctd_data.info()

## Extracting a column

A pandas `Series` can be extracted from a `DataFrame` using one of its columns:

In [ ]:
temp = ctd_data['Temperature']

Some of its attributes:

In [ ]:
print(type(temp))
print(temp.dtype)
print(temp.shape)
print(temp.nbytes)

In [ ]:
print(temp)

Note it has both the index and data coloumn, but the shape is still only 1D. The index is more like a coordinate rather than data in itself.

## Numpy as pandas' backend

It is always possible to fall back to a `numpy` array to pass on to scientific libraries that need them: SciPy, scikit-learn, etc:

In [ ]:
print(ctd_data['Temperature'].values)
print("ctd_data is a", type(ctd_data['Temperature'].values))

## Cleaning data

<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">The truth about data science: cleaning your data is 90% of the work. Fitting the model is easy. Interpreting the results is the other 90%.</p>&mdash; Jake VanderPlas (@jakevdp) <a href="https://twitter.com/jakevdp/status/742406386525446144">June 13, 2016</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

### Renaming columns

Column names in our data don't have any units:

In [ ]:
list(ctd_data)

If we know the units of the variables, we can rename the columns to include these units. This is especailly useful if you plan to carry out unit conversions later:

In [ ]:
ctd_data.columns = ['Depth_m', 'Temperature_C', 'Oxygen_ml/l', 'Irradiance', 'Salinity_psu']
ctd_data.columns

Note the use of underscores `_` in the renaming. It may not be pretty, but putting spaces in variable names will cause problems later on in your processing.

### Deleting columns

Let's drop Irradiance collumn from the data frame:

In [ ]:
ctd_data = ctd_data.drop('Irradiance', 1)

In [ ]:
ctd_data.head()

### Changing index
It would make more sense to have Depth column as the index, as the other variables are expected to vary with depth.

In [ ]:
ctd_data.set_index('Depth_m', inplace=True)
ctd_data

## Basic visualisation

Try calling `plot()` method of the `ctd_data` object:

In [ ]:
ctd_data.plot()

We can explicitly pass nicer labels to legend if needed:

In [ ]:
df = pd.DataFrame({'A':26, 'B':20}, index=['N'])
ax = ctd_data.plot()
ax.legend(["Temperature $\mathrm{(^{\circ}C}$)", "Oxygen (ml/l)", "Salinity (psu)"]);

What happens if we pass `subplots=True` as an argument of the `plot()` method?

In [ ]:
# axes = ctd_data.plot( ... )

It is easy to create other useful plots using `DataFrame`:

In [ ]:
# Use matplotlib to initialise your figure with 2 subplots
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(8,4))

# Make boxplots with specific columns in each subplot
ctd_data.boxplot(ax=ax0, column=['Salinity_psu'])
ctd_data.boxplot(ax=ax1, column=['Oxygen_ml/l'])

plt.show()

More options for visualisation can be looked at [the pandas website](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html).

## Exporting data

One of pandas best features is how it simplifies writing text to a `.csv` (comma seperated value) text file that can be read easily by programs like Excel:

In [ ]:
ctd_data.to_csv(data_dir/'ship_ctd_short_clean.csv', index=True, header=True)

## Statistics

You can print a summary of main statistics for the whole DataFrame:

In [ ]:
ctd_data.describe()

You can still call individual ones like in numpy, e.g.:

In [ ]:
np.mean(ctd_data)

## Computing correlations

Both `Series` and `DataFrames` have a `corr()` method to compute the correlation coefficient.

If series are already grouped into a `DataFrame`, computing all correlation coefficients is trivial:

In [ ]:
ctd_data.corr()

If you want to visualise this correlation matrix, uncomment the following code cell.

In [ ]:
# fig, ax = plt.subplots()
# p = ax.imshow(ctd_data.corr(), interpolation="nearest", cmap='RdBu_r', vmin=-1, vmax=1)
# ax.set_xticks(np.arange(len(ctd_data.corr().columns)))
# ax.set_yticks(np.arange(len(ctd_data.corr().index)))
# ax.set_xticklabels(ctd_data.corr().columns)
# ax.set_yticklabels(ctd_data.corr().index)
# fig.colorbar(p)
# plt.show()

## Rolling operations

Pandas includes functionality to do [rolling means](https://en.wikipedia.org/wiki/Moving_average), sums, and more, given a specified window size. This is very useful for smoothing noisy data.

First we will create some noisy data from our dataset:

In [ ]:
from numpy import random

In [ ]:
noisy_data = ctd_data['Temperature_C']

# Randomise data to make it super noisy
noisy_data = noisy_data * 2*random.rand(len(noisy_data)) + noisy_data**2
print(noisy_data)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7,3))
ctd_data['Temperature_C'].plot(ax=ax[0])
noisy_data.plot(ax=ax[1])
plt.show()

In [ ]:
noisy_roll = noisy_data.rolling(window=50)
noisy_roll

In [ ]:
noisy_roll.mean()

In [ ]:
noisy_data.plot(label="Noisy data")
noisy_roll.mean().plot(label="Rolling mean (window = 50)")
plt.legend(loc="best")
plt.show()

There are other rolling functions, such as [`sum()`](https://pandas.pydata.org/docs/reference/api/pandas.core.window.rolling.Rolling.sum.html), and you can even use the [`win_type`](https://docs.scipy.org/doc/scipy/reference/signal.windows.html#module-scipy.signal.windows) argument to use change the window type.

### Exercise 1

Create a new rolling object from the noisy data and apply the `win_type` argument, and explore the result. 

Two common types are `'triang'` and `'gaussian'`.

Note: for gaussian you'll have to specify a standard deviation argument (written as `std`) 
when you apply the `sum()` or `mean()` function afterwards. 

In [ ]:
## your code goes here

## Creating DataFrames

`DataFrame` can be created manually, by grouping several `Series` together.

We will load two `Series` objects from two `.csv` files and combine them into a `DataFrame`.

Data are monthly values of:
* Southern Oscillation Index (SOI) - "a standardized index based on the observed sea level pressure differences between Tahiti and Darwin, Australia"
* Outgoing Longwave Radiation (OLR) - "a proxy for convective precipitation in the western equatorial Pacific"

Data were downloaded from [NOAA's website](https://www.ncdc.noaa.gov/teleconnections/).

In [ ]:
soi_df = pd.read_csv('../data/soi.csv', skiprows=1, parse_dates=[0], index_col=0, na_values=-999.9,
                     date_parser=lambda x: pd.datetime.strptime(x, '%Y%m'))

olr_df = pd.read_csv('../data/olr.csv', skiprows=1, parse_dates=[0], index_col=0, na_values=-999.9,
                     date_parser=lambda x: pd.datetime.strptime(x, '%Y%m'))

In [ ]:
soi_df.head()

In [ ]:
olr_df.head()

In [ ]:
df = pd.DataFrame({'OLR': olr_df.Value,
                   'SOI': soi_df.Value})

In [ ]:
df.head()

Note that, although the data series do not overlap completely, we can combine them seamlesssly into a dataframe because `pandas` understands datetime objects as the index.

In [ ]:
df.describe()

# Groupby operations

Often, we want to calculate aggregated values across the values of a certain index or column. For example, we can quickly compute monthly averages across all the years in our dataframe as follows:

In [ ]:
df_monthly_means = df.groupby(df.index.month).mean()

# Show the result on a plot
df_monthly_means.plot()
plt.xlabel('Month')
plt.show()

## Ordinary Least Square (OLS) regressions

### Numpy polynomial fitting

In [ ]:
from numpy.polynomial import polynomial as P

In [ ]:
x = df['OLR'].values
y = df['SOI'].values

In [ ]:
idx = np.isfinite(x) & np.isfinite(y)

In [ ]:
coefs, stats = P.polyfit(x[idx], y[idx], 1, full=True)

In [ ]:
y2 = P.polyval(x, coefs)

In [ ]:
plt.plot(x, y, linestyle='', marker='o')
plt.plot(x, y2)
plt.show()

### Sklearn regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
df = df.dropna()
x = df['OLR'].values.reshape(-1, 1)
y = df['SOI'].values.reshape(-1, 1)

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(x, y)
y_pred = model.predict(x) 

In [ ]:
plt.scatter(x, y)
plt.plot(x, y_pred, color="orange")
plt.show()

More examples: https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html

## Extra tutorials

Online tutorials with more in-depth operations used in pandas:

* [Kaggle tutorial](https://www.kaggle.com/learn/pandas)
* [Pandas official website Getting Started](https://pandas.pydata.org/docs/getting_started/index.html#getting-started)

## References
* https://github.com/jonathanrocher/pandas_tutorial
* https://github.com/koldunovn/python_for_geosciences
* http://pandas.pydata.org/pandas-docs/stable/index.html#module-pandas
* http://pandas.pydata.org/pandas-docs/stable/10min.html
* https://towardsdatascience.com/linear-regression-in-6-lines-of-python-5e1d0cd05b8d